# Support Vector Machine

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC

from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import GridSearchCV

## Get the Data

In [ ]:
wine_data = load_wine()

In [ ]:
wine_data.keys()

In [ ]:
wine_data['feature_names']

In [ ]:
df_wine = pd.DataFrame(wine_data['data'], columns=wine_data['feature_names'])
df_wine.info()

In [ ]:
print(wine_data['target_names'])

In [ ]:
set(wine_data['target'])

In [ ]:
df_wine.head()

## Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_wine, wine_data['target'], test_size=0.30, random_state=101)

## Standardize the Variables

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train) 
scaled_X_test = scaler.transform(X_test) 

## SVM model

In [ ]:
model = SVC(C=1.0, gamma = 1.0, kernel = 'rbf')

In [ ]:
model.fit(scaled_X_train,y_train)

In [ ]:
y_pred = model.predict(scaled_X_test)

## Evaluation

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

# Gridsearch - Look for the better hyper parameters

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': [1,0.1,0.01,0.001,0.0001],
    'kernel': ['rbf']
} 

In [ ]:
grid = GridSearchCV(SVC(), param_grid, refit=True)

In [ ]:
grid.fit(scaled_X_train,y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
grid_predictions = grid.predict(scaled_X_test)

In [ ]:
confusion_matrix(y_test,grid_predictions)

In [ ]:
print(classification_report(y_test,grid_predictions))